# Where do Rejected Neurips Papers Go?

### [Neil D. Lawrence](http://inverseprobability.com), University of

Cambridge

### 2021-06-16

**Abstract**: Of the 1,678 papers submitted to the 2014 conference, only
414 were presented at the meeting. In this notebook we trace the fate of
the remaining papers. In revisiting the 2014 NeurIPS experiment, we
explored the fate of over 1000 rejected papers. We found that 680 were
available in some form on line. In this notebook we explore where those
papers were published.

$$
$$

<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!---->
<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!-- The last names to be defined. Should be defined entirely in terms of macros from above-->
<!--

-->

## Where do Rejected Papers Go?

<span class="editsection-bracket" style="">\[</span><span
class="editsection"
style=""><a href="https://github.com/lawrennd/talks/edit/gh-pages/_neurips/includes/where-do-the-rejected-papers-go.md" target="_blank" onclick="ga('send', 'event', 'Edit Page', 'Edit', 'https://github.com/lawrennd/talks/edit/gh-pages/_neurips/includes/where-do-the-rejected-papers-go.md', 13);">edit</a></span><span class="editsection-bracket" style="">\]</span>

One facet that we can explore is what the final fate of papers that are
rejected by the conference is.

Of the 1,678 papers submitted to NeurIPS 2014, only 414 were presented
at the final conference. Here we trace the fate of the rejected papers,
we searched Semantic Scholar for evidence of all 1,264 rejected papers.
We looked for papers with similar titles and where the NeurIPS
submission’s contact author was also in the author list. We were able to
track down 680 papers.

This code analyzes those 680 papers extracting their final publication
venue using the Semantic Scholar API.

In [1]:
%pip install cmtutils

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cmtutils.nipsy as nipsy

 /Users/neil/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning:The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.


In [3]:
import os
import yaml

In [4]:
with open(os.path.join(nipsy.review_store, nipsy.outlet_name_mapping), 'r') as f:
    mapping = yaml.load(f, Loader=yaml.FullLoader)

date = "2021-06-11"

citations = nipsy.load_citation_counts(date=date)
decisions = nipsy.load_decisions()
nipsy.augment_decisions(decisions)
joindf = nipsy.join_decisions_citations(decisions, citations)

joindf['short_venue'] = joindf.venue.replace(mapping)

<iframe src="https://inverseprobability.com/talks/slides/diagrams//neurips/where-do-neurips-papers-go.html" width="600" height="450" allowtransparency="true" frameborder="0">
</iframe>

Figure: <i>Sankey diagram showing the flow of NeurIPS papers through the
system from submission to eventual publication.</i>

Of the 680 papers 177 were only found on arXiv, 76 were found as PDFs
online without a publication venue and 427 were published in other
venues. The outlets that received ten or more papers from this group
were AAAI (72 papers), AISTATS (57 papers), ICML (33 papers), CVPR (17
papers), Later NeurIPS (15 papers), JMLR (14 papers), IJCAI (14 papers),
ICLR (13 papers), UAI (11 papers). Opinion about quality of these
different outlets will vary from individual, but from our perspective
all of these outlets are \`top-tier’ for machine learning and related
areas. Other papers appeared at less prestigious outlets, and citation
scores were also recored for papers that remained available only on
ArXiv. Note that there is likely a bias towards outlets that have a
submission deadline shortly after NeurIPS decisions are public,
e.g. submission deadline for AAAI 2015 was six days after NeurIPS
decisions were sent to authors. AISTATS has a submission deadline one
month after.

A Sankey diagram showing where papers submitted to the conference ended
up is shown below.

In [5]:
import plotly.graph_objects as go

In [6]:
thresh_to_show = 3

label = ['submitted', 'oral', 'spotlight', 'poster', 'reject', '/dev/null']
x = [0.1, 0.3, 0.3, 0.3, 0.3, 0.5]
y = [0.4, 0.95, 0.9, 0.85, 0.3, 0.01]
source = [0, 0, 0, 0, 4]
target = [1, 2, 3, 4, 5]
value = [(joindf['Status']=='Oral').sum(),
         (joindf['Status']=='Spotlight').sum(), 
         (joindf['Status']=='Poster').sum(),
         (joindf['Status']=='Reject').sum(),
        joindf.loc[joindf.reject]['venue'].isna().sum()]

venue_counts = joindf.loc[joindf.reject]['short_venue'].value_counts()
venue_show = venue_counts[venue_counts>=thresh_to_show]
target_val = target[-1]
for venue,count in venue_show.items():
    target_val += 1
    value.append(count)
    source.append(4)
    label.append(venue)
    target.append(target_val)
    if venue=='ArXiv':
        y.append(.15)
        x.append(0.75)
    
    elif venue == 'None':
        y.append(.20)
        x.append(0.75)

    else: 
        y.append(.27)
        x.append(0.8)
    

    
value.append(venue_counts[venue_counts<thresh_to_show].sum())
source.append(4)
label.append('other')
target.append(target_val+1)
x.append(0.85)
y.append(1.0)

link = dict(source = source, target = target, value = value)
node = dict(label=label,
            x = x,
            y = y,
            pad=12)
data=go.Sankey(arrangement = "snap",
                 link = link,
                 node = node)

In [8]:
!mkdir neurips

In [9]:
fig = go.Figure(data=data)
fig.update_layout(template="plotly_dark")
fig.show()
fig.write_html(os.path.join(".", "neurips", "where-do-neurips-papers-go.dark.html"))

In [13]:
fig = go.Figure(data=data,
                layout = go.Layout(width=600,
                height=750))
fig.update_layout(template="plotly", font=dict(
        family="sans serif",
        size=14,
        color="Black"
    ))
fig.show()
fig.write_html(os.path.join(".", "neurips", "where-do-neurips-papers-go.html"))
fig.write_image(os.path.join(".", "neurips", "where-do-neurips-papers-go.svg"))

<img src="https://inverseprobability.com/talks/slides/diagrams//neurips/where-do-neurips-papers-go.svg" class="450" width="600" style="vertical-align:middle;"><iframe src="https://inverseprobability.com/talks/slides/diagrams//neurips/where-do-neurips-papers-go.html" width="600" height="450" allowtransparency="true" frameborder="0">
</iframe>

Figure: <i>Sankey diagram showing the flow of NeurIPS papers through the
system from submission to eventual publication.</i>

## Thanks!

For more information on these subjects and more you might want to check
the following resources.

-   twitter: [@lawrennd](https://twitter.com/lawrennd)
-   podcast: [The Talking Machines](http://thetalkingmachines.com)
-   newspaper: [Guardian Profile
    Page](http://www.theguardian.com/profile/neil-lawrence)
-   blog:
    [http://inverseprobability.com](http://inverseprobability.com/blog.html)

## References